In [1]:
import requests
import time
import re
import csv
import pandas as pd
from bs4 import BeautifulSoup


#The website which we are retrieving the data from
url = 'https://www.godchecker.com/' 

#Connect to the URL and parse HTM
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
response.close()

In [2]:
a_tag = soup.find(id="drop-panel-pantheons").find_all('a') #Retrieving from the box with all mythologies
url_list = str(a_tag).split(',')

In [3]:
myth_url = []
for i in url_list:
    a = i.split('"><span class=')[0].split('godchecker.com/')[1]
    a = "https://www.godchecker.com/" + a + "list-of-names/"
    myth_url.append(a)
myth_url #The list of urls which has all the gods in each mythology

['https://www.godchecker.com/african-mythology/list-of-names/',
 'https://www.godchecker.com/australian-aboriginal-mythology/list-of-names/',
 'https://www.godchecker.com/aztec-mythology/list-of-names/',
 'https://www.godchecker.com/baltic-mythology/list-of-names/',
 'https://www.godchecker.com/buddhist-mythology/list-of-names/',
 'https://www.godchecker.com/canaanite-mythology/list-of-names/',
 'https://www.godchecker.com/caribbean-mythology/list-of-names/',
 'https://www.godchecker.com/celtic-mythology/list-of-names/',
 'https://www.godchecker.com/chinese-mythology/list-of-names/',
 'https://www.godchecker.com/egyptian-mythology/list-of-names/',
 'https://www.godchecker.com/etruscan-mythology/list-of-names/',
 'https://www.godchecker.com/finnish-mythology/list-of-names/',
 'https://www.godchecker.com/germanic-mythology/list-of-names/',
 'https://www.godchecker.com/greek-mythology/list-of-names/',
 'https://www.godchecker.com/hawaiian-mythology/list-of-names/',
 'https://www.godchecke

In [4]:
indiv_urls = []
for i in myth_url:
    index = requests.get(i)
    index_soup = BeautifulSoup(index.text, "html.parser")
    index.close()
    index_links = index_soup.find(id="index-links").find_all('a')
    for j in index_links:
        links = j['href']
        indiv_urls.append(links)
    time.sleep(1) #To prevent overloading the website
    
### takes a while to load

In [5]:
indiv_urls = list(dict.fromkeys(indiv_urls)) # Removing the duplicates
indiv_urls

['https://www.godchecker.com/african-mythology/A-BOL/',
 'https://www.godchecker.com/african-mythology/ABASSI/',
 'https://www.godchecker.com/yoruba-mythology/ABIKU/',
 'https://www.godchecker.com/african-mythology/ABUK/',
 'https://www.godchecker.com/african-mythology/ACHIMI/',
 'https://www.godchecker.com/african-mythology/ANDRIAMBAHOMANANA/',
 'https://www.godchecker.com/african-mythology/ADRO/',
 'https://www.godchecker.com/african-mythology/ADROA/',
 'https://www.godchecker.com/african-mythology/ADROANZI/',
 'https://www.godchecker.com/yoruba-mythology/AGANJU/',
 'https://www.godchecker.com/african-mythology/AGE/',
 'https://www.godchecker.com/african-mythology/AHA-NJOKU/',
 'https://www.godchecker.com/african-mythology/AIGAMUXA/',
 'https://www.godchecker.com/yoruba-mythology/AJE-SHALUGA/',
 'https://www.godchecker.com/african-mythology/AJOK/',
 'https://www.godchecker.com/african-mythology/AKONGO/',
 'https://www.godchecker.com/african-mythology/ALA/',
 'https://www.godchecker.c

In [6]:
len(indiv_urls)

3885

In [7]:
def remove_tags(text):
    """Remove html tags from a string"""
    text =  re.sub('<.*?>', '', text)
    text = re.sub('\[','',text)
    text = re.sub('\]','',text)
    text = re.sub(',','',text)
    return text

def text_to_list(text):
    """Cleaning the text and changing it to a list"""
    text = [x.split(': ') for x in text.split('\n')]
    text = [s for s in text if (s != [''] and s != [' '])]
    return text

In [13]:
df_list = []
"""Retrieving actual data from individual God's site"""
for site in indiv_urls:
    response_site = requests.get(site)
    response_soup = BeautifulSoup(response_site.text, "html.parser")
    box = response_soup.find(class_= "vitalsbox")
    if box is not None:
        facts = box.find_all('p')
        text = remove_tags(str(facts))
        text = text_to_list(text)
        dic = {}
        for k in (text):
            dic[k[0]] = k[1]
        cat = response_soup.find(id="section_header").find('h1')
        dic['Origin'] = remove_tags(str(cat))
        df = pd.DataFrame([dic])
        df_list.append(df)
        df = df[0:0]
        time.sleep(0.3) #Prevent overloading the site

### takes very long to load

In [14]:
data = pd.concat(df_list,sort=False, ignore_index=True) #Converting the data into DataFrame
data

,Name,Pronunciation,Alternative names,Gender,Type,Area or people,Celebration or Feast Day,In charge of,Area of expertise,Good/Evil Rating,Popularity index,Origin,Role,Name means,Associated with,Birth and Death Dates
0,a-Bol,Coming soon,u-Thembra,Female,Goddess,Baga of Guinea,Unknown at present,Fertility,Fertility,GOOD quite approachable,1364,African Mythology,NaN,NaN,NaN,NaN
1,Abassi,Coming soon,,Male,God,Efik people of Nigeria,Unknown at present,Creating,Creation,GOOD quite approachable,1666,African Mythology,NaN,NaN,NaN,NaN
2,Abiku,Coming soon,,Male,Demons,Yoruba / Dahomey people of Nigeria and Benin,Unknown at present,NaN,NaN,TOTALLY EVIL danger!,1509,Yoruba Mythology,Unknown at present,NaN,NaN,NaN
3,Abuk,Coming soon,,Female,Goddess,Dinka people of the Sudan,Unknown at present,Plants and Vegetation,Plants,GOOD quite approachable,981,African Mythology,NaN,NaN,NaN,NaN
4,Achimi,Coming soon,,Female,Goddess,Kabyle people of Algeria,Unknown at present,Buffaloes,Buffaloes,OKAY not bad,972,African Mythology,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3879,Zdislava Berka,Coming soon,,Female,Saint,NaN,Unknown at present,NaN,NaN,Unknown at present,258,Christian mythology,Unknown at present,NaN,NaN,1200-1252
3880,Zeno,Coming soon,Zeno of Verona,Male,Saint,NaN,Unknown at present,NaN,NaN,Unknown at present,297,Christian mythology,Unknown at present,NaN,NaN,???-380
3881,Zenobius of Florence,Coming soon,,Male,Saint,NaN,Unknown at present,NaN,NaN,Unknown at present,296,Christian mythology,Unknown at present,NaN,NaN,310-390
3882,Zita,Coming soon,,Female,Saint,NaN,Unknown at present,NaN,NaN,Unknown at present,254,Christian mythology,Unknown at present,NaN,NaN,1212-1272


In [20]:
data.to_csv('Godchecker_data.csv',index=False) #Export data as CSV file